# House Price Dataset - Cleaning

We will clean our dataset in preparation for data analysis. As a reminder, the dataset is made up of 2181 data points based on the following search criteria:

- Västra Götalands Län
- Villa
- Rooms min 3,5
- min final price 1,750,000kr
- max final price 4,000,000kr
- Last 12 months


https://www.hemnet.se/salda/bostader?item_types[]=villa&location_ids[]=17755&page=1&rooms_min=3.5&selling_price_max=4000000&selling_price_min=1750000 




In [78]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [79]:
pd.set_option('max_rows',600)
df = pd.read_csv('housing_prices.csv')
df.shape

(2181, 10)

In [80]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2181 entries, 0 to 2180
Data columns (total 10 columns):
adress        2181 non-null object
house_type    2181 non-null object
location      2181 non-null object
size          2181 non-null object
tomt          2148 non-null object
biarea        2181 non-null object
sold_price    2181 non-null object
date_sold     2181 non-null object
price_diff    1886 non-null object
agent         2181 non-null object
dtypes: object(10)
memory usage: 170.5+ KB


In [81]:
df.head()

,adress,house_type,location,size,tomt,biarea,sold_price,date_sold,price_diff,agent
0,Västra Mjörnvägen 62,Vinterbonat fritidshus,Vinterbonat fritidshus Tjör...,81 m² 5 rum ...,4 951 m² tomt,Slutpris 3 650 000 kr Såld 2...,Slutpris 3 650 000 kr,Såld 25 juni 2020,"<div class=""sold-property-listing__price-chang...",Gofab Fastighetsbyrå
1,Södra Moränvägen 40,Villa,"Villa Ed, Dals-Ed",219 m² 6 rum...,3 132 m² tomt,64 m² biarea,Slutpris 3 600 000 kr,Såld 24 juni 2020,"<div class=""sold-property-listing__price-chang...",Fastighetsbyrån Dals-Ed/Bengtsfors
2,Röds höjd 134,Villa,"Villa Svenshögen, ...",100 m² 5 rum...,1 685 m² tomt,75 m² biarea,Slutpris 2 450 000 kr,Såld 24 juni 2020,"<div class=""sold-property-listing__price-chang...",Fastighetsbyrån Stenungsund
3,Lugnås Magnilund 1,Villa,"Villa Lugnås, Mari...",79 m² 5 rum ...,27 661 m² tomt,8 m² biarea,Slutpris 1 920 000 kr,Såld 24 juni 2020,"<div class=""sold-property-listing__price-chang...",Fastighetsbyrån Mariestad
4,Sågdammsvägen 9,Villa,"Villa Landvetter, ...",87 m² 4 rum ...,268 m² tomt,Slutpris 3 850 000 kr Såld 2...,Slutpris 3 850 000 kr,Såld 24 juni 2020,"<div class=""sold-property-listing__price-chang...",Fastighetsbyrån Mölnlycke – Härryda


## Initial Findings

Columns are all text and need to be converted to the appropriate datatypes.

- location: needs splitting into 2 columns
- size: needs splitting into 2 columns, size msq and number of rooms
- tomt: convert to an integer
- biarea: integer
- sold_price: convert to integer, replace slutpris, kr etc,
- date sold, remove såld and convert to a suitable format
- price_diff - obs work 

## Price_diff column cleaning





In [82]:
df['price_diff'] = df['price_diff'].str.replace('<div class="sold-property-listing__price-change">', '').replace('</div>', '').replace('\n', '')
df['price_diff'] = df['price_diff'].str.replace('</div>', '')
df['price_diff'] = df['price_diff'].str.replace('\n', '')
df['price_diff'] = df['price_diff'].str.replace('%', '')
df['price_diff'] = df['price_diff'].str.replace('+', '')
df['price_diff'] = df['price_diff'].str.replace(' ', '')
#replace empty values with nan
df['price_diff'] = df['price_diff'].replace(r'^\s*$', np.NaN, regex=True)
#values with no value is because they were sold at the utgångspris
df['price_diff'] = df['price_diff'].fillna(0)
df['price_diff'] = df['price_diff'].astype(int)


In [83]:
#check 
df['price_diff'].head()

0    46
1    13
2    -6
3    28
4     7
Name: price_diff, dtype: int64

## Tomt Column Cleaning

In [84]:
#.str.replace('\xa0', ' ') removes the spaces in the str numbers
df['tomt'] = df['tomt'].str.replace('m²', '').str.replace('tomt','').str.replace('\xa0', ' ')
df['tomt'] = df['tomt'].str.replace(' ','')
df['tomt'] = df['tomt'].fillna(int)


In [85]:
df['tomt'].value_counts(dropna=False)

<class 'int'>    33
868              10
1020              8
810               7
720               7
                 ..
1975              1
4917              1
2135              1
446               1
57159             1
Name: tomt, Length: 1426, dtype: int64

## Sold_price Cleaning

In [86]:
df['sold_price'] = df['sold_price'].str.replace('Slutpris', '').str.replace('kr', '').str.replace('\xa0', ' ').str.replace(' ','')
df['sold_price'] = df['sold_price'].astype(int)


In [87]:
df['sold_price'].value_counts(dropna=False)

2500000    59
3500000    59
3000000    58
2300000    49
2200000    46
2800000    46
2900000    43
2100000    43
3200000    42
3100000    41
2000000    41
1800000    41
4000000    37
3800000    37
2400000    35
2700000    34
1850000    34
3600000    33
3700000    33
1900000    32
2600000    32
3400000    32
3300000    32
2550000    30
2650000    30
3900000    29
2950000    28
2850000    27
2250000    27
1750000    27
3750000    27
3650000    25
3850000    25
2150000    24
2350000    24
3050000    23
3150000    23
3350000    22
2450000    20
2050000    20
3450000    19
2750000    18
3250000    17
1950000    17
1875000    15
3950000    13
3550000    12
2425000    10
2995000    10
2775000     9
1995000     9
3495000     9
3525000     8
2525000     8
1975000     8
2675000     8
3875000     8
2725000     8
1775000     8
3995000     8
2575000     7
2475000     7
2495000     7
2610000     7
3325000     7
2410000     7
2325000     7
3475000     7
3025000     6
2025000     6
3825000     6
207500

In [88]:
df.head()

,adress,house_type,location,size,tomt,biarea,sold_price,date_sold,price_diff,agent
0,Västra Mjörnvägen 62,Vinterbonat fritidshus,Vinterbonat fritidshus Tjör...,81 m² 5 rum ...,4951,Slutpris 3 650 000 kr Såld 2...,3650000,Såld 25 juni 2020,46,Gofab Fastighetsbyrå
1,Södra Moränvägen 40,Villa,"Villa Ed, Dals-Ed",219 m² 6 rum...,3132,64 m² biarea,3600000,Såld 24 juni 2020,13,Fastighetsbyrån Dals-Ed/Bengtsfors
2,Röds höjd 134,Villa,"Villa Svenshögen, ...",100 m² 5 rum...,1685,75 m² biarea,2450000,Såld 24 juni 2020,-6,Fastighetsbyrån Stenungsund
3,Lugnås Magnilund 1,Villa,"Villa Lugnås, Mari...",79 m² 5 rum ...,27661,8 m² biarea,1920000,Såld 24 juni 2020,28,Fastighetsbyrån Mariestad
4,Sågdammsvägen 9,Villa,"Villa Landvetter, ...",87 m² 4 rum ...,268,Slutpris 3 850 000 kr Såld 2...,3850000,Såld 24 juni 2020,7,Fastighetsbyrån Mölnlycke – Härryda


## Size Column Cleaning

In [89]:
df['size'] = df['size'].str.replace('m²', '').str.replace('rum', '').str.replace(' ','').str.replace('kr/mån', '').str.replace(',','.')

In [90]:
#split size column into columns in a new df so we can select the room and size values. expand = True is the parameter which does this.
temp_df = df['size'].str.split(expand=True)

In [91]:
#rename columns and remove the last 2 columns
temp_df.columns = ['size_sq', 'rooms','3','4']
temp_df = temp_df.drop(['3', '4'], axis=1)

In [92]:
df = df.merge(temp_df, how='inner', left_index=True, right_index=True)
df.head(150)

,adress,house_type,location,size,tomt,biarea,sold_price,date_sold,price_diff,agent,size_sq,rooms
0,Västra Mjörnvägen 62,Vinterbonat fritidshus,Vinterbonat fritidshus Tjör...,81 5,4951,Slutpris 3 650 000 kr Såld 2...,3650000,Såld 25 juni 2020,46,Gofab Fastighetsbyrå,81,5
1,Södra Moränvägen 40,Villa,"Villa Ed, Dals-Ed",219 6,3132,64 m² biarea,3600000,Såld 24 juni 2020,13,Fastighetsbyrån Dals-Ed/Bengtsfors,219,6
2,Röds höjd 134,Villa,"Villa Svenshögen, ...",100 5,1685,75 m² biarea,2450000,Såld 24 juni 2020,-6,Fastighetsbyrån Stenungsund,100,5
3,Lugnås Magnilund 1,Villa,"Villa Lugnås, Mari...",79 5,27661,8 m² biarea,1920000,Såld 24 juni 2020,28,Fastighetsbyrån Mariestad,79,5
4,Sågdammsvägen 9,Villa,"Villa Landvetter, ...",87 4,268,Slutpris 3 850 000 kr Såld 2...,3850000,Såld 24 juni 2020,7,Fastighetsbyrån Mölnlycke – Härryda,87,4
5,Nyböle 230,Villa,"Villa Vänersborg, ...",138 6,2810,24 m² biarea,2500000,Såld 24 juni 2020,19,Svensk Fastighetsförmedling Vänersborg,138,6
6,Åsakroken 10,Villa,"Villa Viskafors, B...",128 4,268,Slutpris 2 190 000 kr Såld 2...,2190000,Såld 24 juni 2020,0,Svensk Fastighetsförmedling Borås,128,4
7,Slingervägen 19,Villa,"Villa Hammarsjorden, ...",109 5,608,68 m² biarea,1750000,Såld 24 juni 2020,17,SkandiaMäklarna Hjo/Karlsborg/Skövde/Tibro/Tid...,109,5
8,Staby 260,Villa,"Villa Staby, Kungä...",115 6,1125,90 m² biarea,2650000,Såld 24 juni 2020,18,ERA Axelssons Fastighetsbyrå Kungälv,115,6
9,Gärdesbergen 183,Vinterbonat fritidshus,Vinterbonat fritidshus Tjör...,122 5,1306,17 m² biarea,3700000,Såld 23 juni 2020,16,Fastighetsbyrån Tjörn,122,5


## Location Column Cleaning

In [93]:
df['location'] = df['location'].replace(' ', ',')
temp_df = df['location'].str.split(',', expand=True)

In [94]:
temp_df.columns = ['omrade', 'kommun','1']
temp_df.head()


,omrade,kommun,1
0,Vinterbonat fritidshus Tjör...,None,None
1,Villa Ed,Dals-Ed,None
2,Villa Svenshögen,Stenungsund,None
3,Villa Lugnås,Mariestad,None
4,Villa Landvetter,Härryda,None


In [95]:
df['house_type'].value_counts()
#Remove the following text from column 0

Villa                     2021
Vinterbonat fritidshus      65
Ej specificerad villa       40
Gård                        21
Bostadsrättsvilla           21
Friliggande villa           13
Name: house_type, dtype: int64

In [96]:
temp_df['omrade'] = temp_df['omrade'].str.replace('Villa', '').str.replace('Vinterbonat fritidshus', '').str.replace('Ej specificerad villa', '').str.replace('Bostadsrättsvilla', '').str.replace('Gård', '').str.replace('Friliggande villa', '')
temp_df.head(10)


,omrade,kommun,1
0,Tjörn,None,None
1,Ed,Dals-Ed,None
2,Svenshögen,Stenungsund,None
3,Lugnås,Mariestad,None
4,Landvetter,Härryda,None
5,Vänersborg,Vänersborg,None
6,Viskafors,Borås,None
7,Hammarsjorden,Hjo,None
8,Staby,Kungälv,None
9,Tjörn,None,None


In [97]:
#used a boolean mask to identify error rows and then replaced the values in the omrade and kommun columns.
mask = temp_df['1'].notnull()
temp_df['omrade'][mask] = temp_df['kommun']
temp_df['kommun'][mask] = temp_df['1']

#for rows with nan kommun replace with omrade value
mask_1 = temp_df['kommun'].isnull()
temp_df['kommun'][mask_1] = temp_df['omrade']



In [98]:
# check
# new_df_filt = new_df.dropna()
# new_df_filt.head()

In [99]:
temp_df = temp_df.drop('1', axis=1)
df = df.merge(temp_df, how='inner', left_index=True, right_index=True)
df['kommun'] = df['kommun'].str.strip()
df['omrade'] = df['omrade'].str.strip()
df.head()

,adress,house_type,location,size,tomt,biarea,sold_price,date_sold,price_diff,agent,size_sq,rooms,omrade,kommun
0,Västra Mjörnvägen 62,Vinterbonat fritidshus,Vinterbonat fritidshus Tjör...,81 5,4951,Slutpris 3 650 000 kr Såld 2...,3650000,Såld 25 juni 2020,46,Gofab Fastighetsbyrå,81,5,Tjörn,Tjörn
1,Södra Moränvägen 40,Villa,"Villa Ed, Dals-Ed",219 6,3132,64 m² biarea,3600000,Såld 24 juni 2020,13,Fastighetsbyrån Dals-Ed/Bengtsfors,219,6,Ed,Dals-Ed
2,Röds höjd 134,Villa,"Villa Svenshögen, ...",100 5,1685,75 m² biarea,2450000,Såld 24 juni 2020,-6,Fastighetsbyrån Stenungsund,100,5,Svenshögen,Stenungsund
3,Lugnås Magnilund 1,Villa,"Villa Lugnås, Mari...",79 5,27661,8 m² biarea,1920000,Såld 24 juni 2020,28,Fastighetsbyrån Mariestad,79,5,Lugnås,Mariestad
4,Sågdammsvägen 9,Villa,"Villa Landvetter, ...",87 4,268,Slutpris 3 850 000 kr Såld 2...,3850000,Såld 24 juni 2020,7,Fastighetsbyrån Mölnlycke – Härryda,87,4,Landvetter,Härryda


## Cleaning Agent Column

The agents column is currently showing the local agent's name which isn't of use for data analysis. We're more interested in the overall company i.e Fastighetsbyrån or Svensk Fastighetsförmedling etc. The agents will therefore be grouped by removing the 'local' part of the name in the agent column.

In [100]:
df['agent'].value_counts()

Fastighetsbyrån Uddevalla                                     74
Fastighetsbyrån Borås                                         58
Fastighetsbyrån Kinna                                         54
Fastighetsbyrån Alingsås                                      53
Gofab Fastighetsbyrå                                          47
ERA Garnisonen                                                45
Fastighetsbyrån Lerum                                         40
Bostadsbutiken                                                38
Fastighetsbyrån Vänersborg                                    37
Fastighetsbyrån Falköping                                     36
Svensk Fastighetsförmedling Uddevalla                         35
Svensk Fastighetsförmedling Alingsås                          34
Hus & Hem                                                     34
Länsförsäkringar Fastighetsförmedling Borås                   33
Fastighetsbyrån Trollhättan                                   33
SkandiaMäklarna Hjo/Karls

In [101]:

# # mask = df['agent'].str.contains('Fastighetsbyrån')
# # df['agent'][mask] = 'Fastighetsbyrån'

# #using a mask returned a setting with copy warning due to a chained assignment so using loc instead
# df.loc[df['agent'].str.contains('Fastighetsbyrån'), 'agent'] = 'Fastighetsbyrån'
# using a for loop to make the process quicker

agents = ['Fastighetsbyrån', 'Svensk Fastighetsförmedling', 'Länsförsäkringar Fastighetsförmedling', 'Mäklarhuset', 'ERA Axelssons Fastighetsbyrå', 'Bjurfors', 'SkandiaMäklarna', 'ERA', 'HusmanHagberg']

for agent in agents:
    df.loc[df['agent'].str.contains(agent), 'agent'] = agent
    
df['agent'].value_counts()    

Fastighetsbyrån                           884
Svensk Fastighetsförmedling               443
Länsförsäkringar Fastighetsförmedling     229
Mäklarhuset                               172
ERA                                       123
SkandiaMäklarna                            53
Gofab Fastighetsbyrå                       47
Bostadsbutiken                             38
Hus & Hem                                  34
Bjurfors                                   22
HusmanHagberg                              21
Broberg Mäkleri AB                         20
Lysekilsmäklarna AB                        14
Askengren Fastighetsmäklare                12
Erik Olsson Fastighetsförmedling           11
Bratting Mäklarbyrå                         6
Skara Fastighetsbyrå                        6
Lisenstedt Fastighetsförmedling AB          6
Bostada                                     5
Parder & Veres AB                           4
Lundin Fastighetsbyrå                       3
RE/MAX Warberg & Co               

In [102]:
df.head()

,adress,house_type,location,size,tomt,biarea,sold_price,date_sold,price_diff,agent,size_sq,rooms,omrade,kommun
0,Västra Mjörnvägen 62,Vinterbonat fritidshus,Vinterbonat fritidshus Tjör...,81 5,4951,Slutpris 3 650 000 kr Såld 2...,3650000,Såld 25 juni 2020,46,Gofab Fastighetsbyrå,81,5,Tjörn,Tjörn
1,Södra Moränvägen 40,Villa,"Villa Ed, Dals-Ed",219 6,3132,64 m² biarea,3600000,Såld 24 juni 2020,13,Fastighetsbyrån,219,6,Ed,Dals-Ed
2,Röds höjd 134,Villa,"Villa Svenshögen, ...",100 5,1685,75 m² biarea,2450000,Såld 24 juni 2020,-6,Fastighetsbyrån,100,5,Svenshögen,Stenungsund
3,Lugnås Magnilund 1,Villa,"Villa Lugnås, Mari...",79 5,27661,8 m² biarea,1920000,Såld 24 juni 2020,28,Fastighetsbyrån,79,5,Lugnås,Mariestad
4,Sågdammsvägen 9,Villa,"Villa Landvetter, ...",87 4,268,Slutpris 3 850 000 kr Såld 2...,3850000,Såld 24 juni 2020,7,Fastighetsbyrån,87,4,Landvetter,Härryda


In [103]:
df['biarea'].value_counts(dropna=False)

          60 m² biarea                                                  42
          70 m² biarea                                                  36
          20 m² biarea                                                  36
          80 m² biarea                                                  34
          30 m² biarea                                                  32
                                                                        ..
        Slutpris 3 400 000 kr           Såld 20 mars 2020                1
        Slutpris 2 250 000 kr           Såld 15 juni 2020                1
        Slutpris 3 750 000 kr           Såld 21 maj 2020                 1
        Slutpris 2 200 000 kr           Såld 31 augusti 2019             1
        Slutpris 1 870 000 kr           Såld 22 oktober 2019             1
Name: biarea, Length: 790, dtype: int64

In [104]:
#cleaned the values by removing msq and 'biarea' whilst also changing the rows without data to np.nan
df['biarea'] = df['biarea'].str.replace('m² biarea', '')
df.loc[df['biarea'].str.contains('Slutpris'), 'biarea'] = np.nan


In [105]:
df.head()

,adress,house_type,location,size,tomt,biarea,sold_price,date_sold,price_diff,agent,size_sq,rooms,omrade,kommun
0,Västra Mjörnvägen 62,Vinterbonat fritidshus,Vinterbonat fritidshus Tjör...,81 5,4951,NaN,3650000,Såld 25 juni 2020,46,Gofab Fastighetsbyrå,81,5,Tjörn,Tjörn
1,Södra Moränvägen 40,Villa,"Villa Ed, Dals-Ed",219 6,3132,64,3600000,Såld 24 juni 2020,13,Fastighetsbyrån,219,6,Ed,Dals-Ed
2,Röds höjd 134,Villa,"Villa Svenshögen, ...",100 5,1685,75,2450000,Såld 24 juni 2020,-6,Fastighetsbyrån,100,5,Svenshögen,Stenungsund
3,Lugnås Magnilund 1,Villa,"Villa Lugnås, Mari...",79 5,27661,8,1920000,Såld 24 juni 2020,28,Fastighetsbyrån,79,5,Lugnås,Mariestad
4,Sågdammsvägen 9,Villa,"Villa Landvetter, ...",87 4,268,NaN,3850000,Såld 24 juni 2020,7,Fastighetsbyrån,87,4,Landvetter,Härryda


In [106]:
df['date_sold'] = df['date_sold'].str.replace('Såld', '').str.strip()


#cannot convert dates to a suitable format when they are in Swedish, converted the months to english and then pd.to_datetime()
months_swe = ['januari', 'februari', 'mars', 'april', 'maj', 'juni', 'juli', 'augusti', 'september', 'oktober', 'november', 'december']
months_eng = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']

for month_swe, month_eng in zip(months_swe, months_eng):
    df['date_sold'] = df['date_sold'].str.replace(month_swe, month_eng)
    
df.head()    
    
    

,adress,house_type,location,size,tomt,biarea,sold_price,date_sold,price_diff,agent,size_sq,rooms,omrade,kommun
0,Västra Mjörnvägen 62,Vinterbonat fritidshus,Vinterbonat fritidshus Tjör...,81 5,4951,NaN,3650000,25 June 2020,46,Gofab Fastighetsbyrå,81,5,Tjörn,Tjörn
1,Södra Moränvägen 40,Villa,"Villa Ed, Dals-Ed",219 6,3132,64,3600000,24 June 2020,13,Fastighetsbyrån,219,6,Ed,Dals-Ed
2,Röds höjd 134,Villa,"Villa Svenshögen, ...",100 5,1685,75,2450000,24 June 2020,-6,Fastighetsbyrån,100,5,Svenshögen,Stenungsund
3,Lugnås Magnilund 1,Villa,"Villa Lugnås, Mari...",79 5,27661,8,1920000,24 June 2020,28,Fastighetsbyrån,79,5,Lugnås,Mariestad
4,Sågdammsvägen 9,Villa,"Villa Landvetter, ...",87 4,268,NaN,3850000,24 June 2020,7,Fastighetsbyrån,87,4,Landvetter,Härryda


## Cleaning The date_sold Column

In [107]:
df['date_sold'] = pd.to_datetime(df['date_sold'], format='%d %B %Y', errors='coerce')


In [108]:
df.tail(250)

,adress,house_type,location,size,tomt,biarea,sold_price,date_sold,price_diff,agent,size_sq,rooms,omrade,kommun
1931,Hammarhöjden 650,Villa,"Villa Kode, Kungäl...",68 4,1157,NaN,2650000,2019-08-22,6,Svensk Fastighetsförmedling,68,4,Kode,Kungälv
1932,Hagen 14,Vinterbonat fritidshus,Vinterbonat fritidshus ...,61 4,828,NaN,3500000,2019-08-22,0,Mäklarhuset,61,4,Sydkoster,Strömstad
1933,Ålyckegatan 1,Villa,"Villa Ljunghed, Li...",170 5,906,NaN,2700000,2019-08-22,20,Mäklarhuset,170,5,Ljunghed,Lidköping
1934,Sofielundsgatan 36,Villa,"Villa Mösseberg, F...",129 4,378,NaN,2725000,2019-08-22,4,Mäklarhuset,129,4,Mösseberg,Falköping
1935,Ingatorp Heden 1,Villa,"Villa Sätuna, Falk...",100 4,10155,NaN,2110000,2019-08-21,59,Fastighetsbyrån,100,4,Sätuna,Falköping
1936,Vråkvägen 5,Villa,"Villa Foss, Munked...",123 5,747,20,2700000,2019-08-21,23,Fastighetsbyrån,123,5,Foss,Munkedal
1937,Länsmansgatan 20,Villa,"Villa Norrmalm, Sk...",135 7,832,48,3250000,2019-08-21,14,Bratting Mäklarbyrå,135,7,Norrmalm,Skövde
1938,Hulten Petsbo,Villa,"Villa Mark, Mark",74 5,1011,74,2560000,2019-08-21,16,Svensk Fastighetsförmedling,74,5,Mark,Mark
1939,Bäckahästens väg 55,Bostadsrättsvilla,Bostadsrättsvilla Älvän...,101 4 4 586,<class 'int'>,NaN,3000000,2019-08-21,2,Länsförsäkringar Fastighetsförmedling,101,4,Älvängen,Ale
1940,Olserödsgatan 16,Villa,"Villa Olseröd, Kun...",118 5,238,NaN,3875000,2019-08-21,2,Länsförsäkringar Fastighetsförmedling,118,5,Olseröd,Kungälv


In [109]:
df = df.drop(['location', 'size', 'tomt'], axis=1)
df.head()

,adress,house_type,biarea,sold_price,date_sold,price_diff,agent,size_sq,rooms,omrade,kommun
0,Västra Mjörnvägen 62,Vinterbonat fritidshus,NaN,3650000,2020-06-25,46,Gofab Fastighetsbyrå,81,5,Tjörn,Tjörn
1,Södra Moränvägen 40,Villa,64,3600000,2020-06-24,13,Fastighetsbyrån,219,6,Ed,Dals-Ed
2,Röds höjd 134,Villa,75,2450000,2020-06-24,-6,Fastighetsbyrån,100,5,Svenshögen,Stenungsund
3,Lugnås Magnilund 1,Villa,8,1920000,2020-06-24,28,Fastighetsbyrån,79,5,Lugnås,Mariestad
4,Sågdammsvägen 9,Villa,NaN,3850000,2020-06-24,7,Fastighetsbyrån,87,4,Landvetter,Härryda


## Adding The Utgangspris

In order to get the utgångspriser we need to use both the sold price and price diff columns.

Having worked out the utgångspris we can see that the price_diff column has been rounded which means that we don't have the actually utgångspris, but we can get around this by rounding the values. The pandas round function on works with base 10 so I've used a custom function from stackoverflow to get around this. 

Sadly this will never be perfect and there will me small rounding errors with the utgangspris column. We will continue with this column until the analysis and drop the column if it isn't of use to us. As we already have the price_diff column, this shouldn't be an issue. It is worth noting in the read me though that the price_diff % is rounded and as such affects the reliability of the data. As it is rounded for all values though, this isn't a big issue and doesn't dramatically affect the quality of our data.

In [110]:
df['utgangspris'] = df['sold_price'] / (1+(df['price_diff']/100))
df['utgangspris'] = df['utgangspris'].astype(int)
# df['utgangspris'] = df['utgangspris'].round(-5)

df.head()

,adress,house_type,biarea,sold_price,date_sold,price_diff,agent,size_sq,rooms,omrade,kommun,utgangspris
0,Västra Mjörnvägen 62,Vinterbonat fritidshus,NaN,3650000,2020-06-25,46,Gofab Fastighetsbyrå,81,5,Tjörn,Tjörn,2500000
1,Södra Moränvägen 40,Villa,64,3600000,2020-06-24,13,Fastighetsbyrån,219,6,Ed,Dals-Ed,3185840
2,Röds höjd 134,Villa,75,2450000,2020-06-24,-6,Fastighetsbyrån,100,5,Svenshögen,Stenungsund,2606382
3,Lugnås Magnilund 1,Villa,8,1920000,2020-06-24,28,Fastighetsbyrån,79,5,Lugnås,Mariestad,1500000
4,Sågdammsvägen 9,Villa,NaN,3850000,2020-06-24,7,Fastighetsbyrån,87,4,Landvetter,Härryda,3598130


In [111]:
def custom_round(x, base=5):
    return int(base * round(float(x)/base))

df['utgangspris'] = df['utgangspris'].apply(lambda x: custom_round(x, base=25000))

df.head()
    

,adress,house_type,biarea,sold_price,date_sold,price_diff,agent,size_sq,rooms,omrade,kommun,utgangspris
0,Västra Mjörnvägen 62,Vinterbonat fritidshus,NaN,3650000,2020-06-25,46,Gofab Fastighetsbyrå,81,5,Tjörn,Tjörn,2500000
1,Södra Moränvägen 40,Villa,64,3600000,2020-06-24,13,Fastighetsbyrån,219,6,Ed,Dals-Ed,3175000
2,Röds höjd 134,Villa,75,2450000,2020-06-24,-6,Fastighetsbyrån,100,5,Svenshögen,Stenungsund,2600000
3,Lugnås Magnilund 1,Villa,8,1920000,2020-06-24,28,Fastighetsbyrån,79,5,Lugnås,Mariestad,1500000
4,Sågdammsvägen 9,Villa,NaN,3850000,2020-06-24,7,Fastighetsbyrån,87,4,Landvetter,Härryda,3600000


## Changing Column Names

In [112]:
df.columns

Index(['adress', 'house_type', 'biarea', 'sold_price', 'date_sold',
       'price_diff', 'agent', 'size_sq', 'rooms', 'omrade', 'kommun',
       'utgangspris'],
      dtype='object')

In [113]:
df.columns = ['adress', 'house_type', 'biarea_msq', 'sold_price_kr', 'date_sold',
       'price_diff_%', 'agent', 'size_msq', 'rooms', 'omrade', 'kommun',
       'utgangspris_kr']

df.head()

,adress,house_type,biarea_msq,sold_price_kr,date_sold,price_diff_%,agent,size_msq,rooms,omrade,kommun,utgangspris_kr
0,Västra Mjörnvägen 62,Vinterbonat fritidshus,NaN,3650000,2020-06-25,46,Gofab Fastighetsbyrå,81,5,Tjörn,Tjörn,2500000
1,Södra Moränvägen 40,Villa,64,3600000,2020-06-24,13,Fastighetsbyrån,219,6,Ed,Dals-Ed,3175000
2,Röds höjd 134,Villa,75,2450000,2020-06-24,-6,Fastighetsbyrån,100,5,Svenshögen,Stenungsund,2600000
3,Lugnås Magnilund 1,Villa,8,1920000,2020-06-24,28,Fastighetsbyrån,79,5,Lugnås,Mariestad,1500000
4,Sågdammsvägen 9,Villa,NaN,3850000,2020-06-24,7,Fastighetsbyrån,87,4,Landvetter,Härryda,3600000


In [114]:
df.to_csv('house_prices_cleaned.csv')

In [115]:
df= pd.read_csv('house_prices_cleaned.csv',index_col=0)
df.head()

,adress,house_type,biarea_msq,sold_price_kr,date_sold,price_diff_%,agent,size_msq,rooms,omrade,kommun,utgangspris_kr
0,Västra Mjörnvägen 62,Vinterbonat fritidshus,NaN,3650000,2020-06-25,46,Gofab Fastighetsbyrå,81.0,5.0,Tjörn,Tjörn,2500000
1,Södra Moränvägen 40,Villa,64,3600000,2020-06-24,13,Fastighetsbyrån,219.0,6.0,Ed,Dals-Ed,3175000
2,Röds höjd 134,Villa,75,2450000,2020-06-24,-6,Fastighetsbyrån,100.0,5.0,Svenshögen,Stenungsund,2600000
3,Lugnås Magnilund 1,Villa,8,1920000,2020-06-24,28,Fastighetsbyrån,79.0,5.0,Lugnås,Mariestad,1500000
4,Sågdammsvägen 9,Villa,NaN,3850000,2020-06-24,7,Fastighetsbyrån,87.0,4.0,Landvetter,Härryda,3600000
